# Convolution via Toeplitz Matrices in PyTorch

In this notebook, we explore how **discrete convolutions** (like those in neural networks) can be expressed as **ordinary matrix multiplications** when we unroll the sliding-window operation into **Toeplitz** (or Toeplitz-like) matrices.

We'll do this in **two parts**:
- **Part 1**: 1D convolution
- **Part 2**: 2D convolution

Additionally, we will discuss:
- **Weight Sharing**: Convolution weights are repeated (shifted) in a Toeplitz matrix, drastically reducing the number of parameters vs. a fully dense matrix.
- **Translation Equivariance**: A key property of convolution is that if the input shifts, the output shifts in the same way (up to boundary effects), unlike a generic matrix multiplication.

## Why Toeplitz?
*Convolution* from a vector/matrix viewpoint is a linear transformation with a very particular structure. For a 1D kernel of length \(K\), we form a matrix whose rows are shifted copies of the kernel. In 2D, the idea extends to *block-Toeplitz* matrices. This structure imposes **weight sharing**—many entries of the large matrix are the **same** parameter. This reduces the degrees of freedom and imparts **equivariance** to shifts.

---

# Part 1: 1D Convolution

In this section:
- We'll use **PyTorch** to define a simple 1D convolution layer with a known kernel size.
- We'll see that the convolution can be replicated by building a **Toeplitz matrix** that multiplies the input vector directly.

## 1.1 PyTorch 1D Convolution

We'll create:
- An input signal of length \(N=6\) (just for demonstration).
- A convolution kernel of length \(K=3\).
- A PyTorch conv layer (`nn.Conv1d`) with **1 input channel** and **1 output channel**, kernel size=3.

Then, we will feed the input into `conv1d` and see the output. After that, we'll replicate the exact same operation by building and applying the Toeplitz matrix manually.


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

# Fix a seed for reproducibility
torch.manual_seed(42)

# Create a 1D input of length 6.
# We'll treat it as [batch_size=1, in_channels=1, length=6].
x_1d = torch.tensor([[ [1.0, 3.0, 2.0, 1.0, 0.0, -1.0] ]])
print("x_1d shape:", x_1d.shape)  # (1, 1, 6)

# Create a conv1d layer with 1 input channel, 1 output channel, kernel_size=3.
conv1d_layer = nn.Conv1d(in_channels=1, out_channels=1, kernel_size=3, bias=False)

# Let's see how many parameters:
print("Conv1D weight shape:", conv1d_layer.weight.shape)  # should be (1,1,3)

# Forward pass
y_1d_torch = conv1d_layer(x_1d)
print("Output shape:", y_1d_torch.shape)
print("Output:", y_1d_torch)

# We'll extract the weights (kernel) for demonstration.
kernel_1d = conv1d_layer.weight.detach().clone().flatten()
print("\nLearned kernel (random init):", kernel_1d)

### 1.2 Building the Toeplitz Matrix

A 1D convolution with kernel size \(K=3\) and input length \(N=6\) (valid convolution) yields an output of length \(N - K + 1 = 4\).  
We can express this as:
\[
y = T\, x\]
where \(T\) is a \(4\times 6\) matrix containing shifts of the (learned) kernel.

Here's the formula for valid 1D convolution output index \(n\):
\[
y[n] = \sum_{k=0}^{K-1} w[k]\, x[n + k], \quad n = 0,1,2,..., N-K.
\]
So row \(n\) of \(T\) is basically `[0..0, w[0], w[1], w[2], 0..0]` aligned to produce that sum.

### 1.3 Exercise
**Task**: In the code below, we provide a function to build the Toeplitz matrix for a given kernel of length \(K\), intended for input length \(N\).  
Your job is to verify/correct the logic so that multiplying by the flattened input produces the **same** output as `conv1d_layer(x_1d)`.

> **Hint**: The shape of the Toeplitz matrix here should be `(N-K+1) x N`. Make sure to place the kernel coefficients in the correct columns for each row.


In [ ]:
def build_toeplitz_1d(kernel, N):
    """
    Build a (N - K + 1) x N toeplitz matrix from a 1D kernel.
    kernel: 1D tensor of shape [K]
    N: total input length.
    """
    K = kernel.shape[0]
    out_len = N - K + 1
    T = torch.zeros(out_len, N)

    # Fill T so that row n corresponds to kernel placed starting at column n.
    # e.g. T[n, n] = w[0], T[n, n+1] = w[1], ...
    for n in range(out_len):
        for k in range(K):
            T[n, n + k] = kernel[k]

    return T

# Build the Toeplitz matrix from the learned kernel.
K_1d = kernel_1d.shape[0]  # 3
N_1d = x_1d.shape[-1]      # 6
T_1d = build_toeplitz_1d(kernel_1d, N_1d)

print("Toeplitz matrix shape:", T_1d.shape)
print(T_1d)

# Flatten input x_1d to shape (N,)
# x_1d is (1,1,6). We'll remove the batch & channel dims.
x_1d_flat = x_1d[0,0,:]
print("\nFlattened input:", x_1d_flat)

# Multiply:
y_1d_manual = T_1d @ x_1d_flat
print("\nManual Toeplitz-based output:", y_1d_manual)

# Compare with the PyTorch conv1d output (which is shape (1,1,4)).
print("\nPyTorch output:", y_1d_torch)
print("\nDifference:", y_1d_manual - y_1d_torch.flatten())

If everything is correct, the difference should be nearly zero (floating-point variations aside). If not, adjust the Toeplitz construction code.

### 1.4 Weight Sharing and Degrees of Freedom

1. **A fully dense \(4\times 6\) matrix** (mapping 6 inputs to 4 outputs) has \(4 \times 6 = 24\) parameters.  
2. **Our Toeplitz matrix** is defined by a kernel of length \(K=3\). That is only **3 parameters**. All other entries of the matrix are just copies of these parameters (with appropriate shifts).

So, a generic linear map from 6 inputs to 4 outputs would have 24 degrees of freedom, but the convolution structure reduces that to only 3 (plus a bias if we included it). This is **weight sharing**.

> **Exercise**: Suppose we had an input of length \(N=100\) and a kernel of length \(K=5\). Then the output length is 96. How many parameters does a fully dense transformation (96 x 100) have vs. a Toeplitz-based convolution? Fill in:
- Fully dense: ??
- Toeplitz-based: ??

### 1.5 Equivariance

Convolution has a property called **translation equivariance**: shifting the input by some amount shifts the output by the **same** amount (ignoring boundary effects). In other words, the filter "responds" in the same way at every location, which is precisely why we can share the same kernel parameters across positions.

In contrast, a generic matrix with no repeated entries does *not* produce the same shift in output if you shift the input.

----

# Part 2: 2D Convolution

Now let's do a small 2D case. We'll create:
- A small "image" (5x5) with a noticeable pattern.
- A 2D filter (3x3) that might detect some feature (e.g., a cross or edges).
- We'll apply `nn.Conv2d(..., kernel_size=3, bias=False)` in PyTorch, then replicate it via a Toeplitz-like matrix multiplication.

## 2.1 Example Image

We'll make a small 5x5 image with a **plus sign** shape in the center. This will help us see if a certain filter detects edges or corners.

```
0 0 1 0 0
0 0 1 0 0
1 1 1 1 1
0 0 1 0 0
0 0 1 0 0
```

### 2.2 PyTorch Conv2D
- 1 input channel, 1 output channel, kernel_size=3.
- We'll examine the learned 3x3 kernel and compare results.


In [ ]:
# Construct the 5x5 image
image_2d = torch.tensor([
    [0, 0, 1, 0, 0],
    [0, 0, 1, 0, 0],
    [1, 1, 1, 1, 1],
    [0, 0, 1, 0, 0],
    [0, 0, 1, 0, 0]
], dtype=torch.float)

# Reshape to (batch=1, channels=1, height=5, width=5)
x_2d = image_2d.unsqueeze(0).unsqueeze(0)  # shape (1,1,5,5)
print("x_2d shape:", x_2d.shape)

# Create a 2D conv layer
conv2d_layer = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=3, bias=False)

# Forward pass
y_2d_torch = conv2d_layer(x_2d)
print("Output shape:", y_2d_torch.shape)  # (1,1,3,3) for valid padding
print("Output:", y_2d_torch)

# The learned kernel, shape (1,1,3,3)
kernel_2d = conv2d_layer.weight.detach().clone()[0,0,:,:]
print("\nLearned 3x3 kernel:")
print(kernel_2d)

### 2.3 Building the 2D Toeplitz-Like Matrix

For a **valid** 2D convolution of a \(5\times 5\) input with a \(3\times 3\) kernel, the output is \((5-3+1)\times (5-3+1) = 3\times 3\). When flattened in row-major order, that's 9 output values.

If we also flatten the input in row-major order (25 values), then we can create a \(9\times 25\) matrix \(T\) that implements the same mapping.

The rule for valid 2D convolution:
\[
Y[i,j] = \sum_{r=0}^{2}\sum_{s=0}^{2} K[r,s]\, X[i + r, j + s],
\]
with \(i,j\) from 0..2.

### Exercise
**Task**: Complete the code below that attempts to build such a Toeplitz-like matrix. Then verify that multiplying by the flattened input yields the same 3x3 result as PyTorch's conv2d layer.**

> **Hint**: The indexing can be tricky. Flatten the input in row-major order: input pixel (row, col) maps to `row * width + col`. The output (i, j) can be mapped to `i * out_width + j` in the rows of the big matrix.

> **Reflection**: This matrix can have many repeated entries of the same kernel value—**weight sharing**. A generic 9x25 matrix would have 225 free parameters, but our 3x3 kernel has only 9. This is a huge reduction!


In [ ]:
def build_toeplitz_2d(kernel_2d, H, W):
    """
    Builds a toeplitz-like matrix T of shape (out_H*out_W, H*W),
    so that T * vec(X) = vec(Y) for the valid conv.
    kernel_2d: shape (3,3)
    H,W = 5,5 for this example.
    """
    R, S = kernel_2d.shape  # should be 3,3
    out_H = H - R + 1       # 3
    out_W = W - S + 1       # 3
    T = torch.zeros(out_H*out_W, H*W)

    for i in range(out_H):
        for j in range(out_W):
            out_idx = i*out_W + j  # flatten (i,j)
            # For each kernel element
            for r in range(R):
                for s in range(S):
                    # The input pixel's row,col
                    in_row = i + r
                    in_col = j + s
                    in_idx = in_row * W + in_col  # flatten (in_row, in_col)
                    T[out_idx, in_idx] = kernel_2d[r,s]

    return T

H_2d, W_2d = 5, 5
T_2d = build_toeplitz_2d(kernel_2d, H_2d, W_2d)
print("T_2d shape:", T_2d.shape)

# Flatten the input image:
x_2d_flat = x_2d[0,0,:,:].flatten()  # shape [25]

# Multiply:
y_2d_manual_flat = T_2d @ x_2d_flat  # shape [9]

# Reshape to (3,3)
y_2d_manual = y_2d_manual_flat.view(3,3)
print("\nManual 2D Toeplitz-based output:")
print(y_2d_manual)

print("\nPyTorch conv2d output:")
print(y_2d_torch[0,0,:,:])  # shape (3,3)

print("\nDifference:", y_2d_manual - y_2d_torch[0,0,:,:])

If the difference is near zero, then the Toeplitz-like matrix approach matches the PyTorch convolution. If not, adjust your indexing.

## 2.4 Weight Sharing and Equivariance in 2D
- A **generic** 9x25 matrix (for mapping a 5x5 image to a 3x3 output) has 225 parameters.
- Our 3x3 kernel has **only 9 parameters**, repeated in the appropriate places.

This weight sharing enforces **translation equivariance** in 2D: shifting the image left/right/up/down leads to a corresponding shift in the output feature map (again, ignoring boundaries).

> **Exercise**: Suppose you had a 10x10 image and a 3x3 filter. The valid output is 8x8 = 64. A generic 64x100 matrix has 6400 parameters. But the 3x3 filter has only 9 parameters. That is a massive reduction in degrees of freedom.

---
# Summary
We have shown that **convolution** in both 1D and 2D can be expressed as a **matrix multiplication**. The matrix is **Toeplitz-like**, meaning it has repeated entries corresponding to the same filter (kernel) weights. This structure:
- **Reduces** the number of parameters drastically (weight sharing).
- Imparts **translation equivariance**: shifting inputs shifts outputs correspondingly.

In contrast, a fully dense matrix has no repeated entries, so it **does not** share weights across positions and does **not** enforce shift-equivariance.

## Further Questions
1. **Implementation**: Real deep-learning frameworks usually do not build this huge matrix. Instead, they use either specialized GPU kernels or the `im2col` trick plus highly optimized matrix multiplication.
2. **Strided** or **padded** convolutions can also be seen in the Toeplitz framework, but the matrix layout changes.
3. **Equivariance** is approximate near boundaries (due to padding choices). Perfect equivariance in the interior of the image is a key property that makes CNNs so effective for translation-invariant tasks like image recognition.

---
## End of Notebook

You can now explore or modify the code, try different kernel sizes, or see how bias terms add an extra row/column in the matrix representation. Enjoy!